# GCN prediction 

- Not used in the current draft.
- Demonstrate the importance of width and depth.
- Demonstrate the predictive power of weak mobility ties, as opposed to spatial and strong ties.
- Test the robustness by varying other GCN components

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import copy
import scipy.sparse as sp
from scipy.sparse import csr_matrix

# GNN packages
# Question: How to use a virtual environment in a jupyter notebook? Mine does not work yet.

from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
from torch_geometric.nn import HeteroConv, GCNConv, Linear, GATConv, SAGEConv


Bad key text.latex.unicode in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key text.latex.preview in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 125 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 157 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')


In [2]:
import sys
sys.path.append("../")
import utils

In [3]:
import importlib
importlib.reload(utils)

<module 'utils' from '../utils.py'>

In [4]:
# read files
with open("../../data/02_intermediate/greater_boston_ct_shp.pickle", 'rb') as f:
    df_shp = pickle.load(f)

with open("../../data/03_processed/greater_boston_ct_socioecon_shp.pickle", 'rb') as f:
    df_socioecon_shp = pickle.load(f)
    
with open("../../data/02_intermediate/boston_stays.pickle", 'rb') as f:
    df = pickle.load(f)

with open("../../data/03_processed/A_home_activity_unweighted_dic.pickle", 'rb') as f:
    A_home_activity_unweighted_dic = pickle.load(f)

with open("../../data/03_processed/A_home_activity_weighted_dic.pickle", 'rb') as f:
    A_home_activity_weighted_dic = pickle.load(f)
    
with open("../../data/05_model_outputs/lasso_coefficients.pickle", 'rb') as f:
    lasso_coef = pickle.load(f)
    

In [80]:
with open("../../data/03_processed/spatial_network_dic.pickle", 'rb') as f:
    spatial_network_dic = pickle.load(f)

In [5]:
# read files
with open("../../data/03_processed/A_home_total_unweighted_dic.pickle", 'rb') as f:
    A_home_total_unweighted_dic = pickle.load(f)

with open("../../data/03_processed/A_home_total_weighted_dic.pickle", 'rb') as f:
    A_home_total_weighted_dic = pickle.load(f)
    

In [6]:
spatial_network_dic.keys()

dict_keys(['boston_queen_contiguity_adj_df', 'boston_rook_contiguity_adj_df', 'boston_5nn_contiguity_adj_df', 'county_adj_df'])

In [7]:
# get the significant activities
sig_activities = list(lasso_coef['inc_per_capita_2018']['lasso (no socio-demographics)'].index)
print(len(sig_activities))
print(sig_activities)


18
['Caribbean', 'Latin American', 'Spanish', 'Auto Garage', 'Fried Chicken', 'Poutine Place', 'Fishing Store', 'Meze Restaurants', 'Food Stand', 'Watch Shops', 'Tapas', 'Peking Duck', 'Drugstore', 'Juice Bar', 'Salad', 'Cycle Studio', 'New American', 'French']


In [8]:
# get the overall activity ranking
activity_counts = np.unique(df.cat, return_counts = True)
print("Total number of activity categories", len(activity_counts[0]))

# 
activity_count_df = pd.DataFrame(activity_counts[1],
                                 index = activity_counts[0],
                                 columns = ['count'])
sorted_activity_count_df = activity_count_df.sort_values('count', ascending=False)
# 
print("Top 20 large-quantity mobility ties: ", list(activity_count_df.sort_values('count', ascending=False).index[:20]))


Total number of activity categories 633
Top 20 large-quantity mobility ties:  ['Office', 'Residential', 'Building', 'Automotive', 'Church', 'Salon / Barbershop', "Doctor's Office", 'Pizza', 'American', 'Gas Station', 'Gym', 'Road', 'Bank', 'Donuts', 'Hardware', 'Grocery Store', 'Supermarket', 'Bar', "Dentist's Office", 'Convenience Store']


In [19]:
# init performance
gcn_performance_dic = {}

# Node-only GCN without edges

In [20]:
# create the baseline regressions
input_vars = ['pop_total_2016',
              'pop_density_2016',
              'age_median_2016', 
              'household_size_avg_2016',
              'sex_male_ratio_2016', 
              'race_white_ratio_2016',
              'edu_bachelor_ratio_2016',
              'edu_master_ratio_2016',
              'edu_master_ratio_2016', 
              'vehicle_per_capita_2016']

# predict inc_per_capita_2018
output_var = 'inc_per_capita_2018'


X = normalize(df_socioecon_shp[input_vars].values, axis = 0)
y = normalize(df_socioecon_shp[output_var].values.reshape(-1, 1), axis = 0).reshape(-1, 1)

X = X.astype('float32')
y = y.astype('float32')

X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y)

X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = \
    train_test_split(X_tensor, y_tensor, test_size = 0.33, random_state = 42)


In [21]:
# model setup: device, model, inputs and outputs.
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model = Linear(10, 1, bias = True).to(device)
X_train_tensor = X_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

# X_tensor = X_tensor.to(device)
# y_tensor = y_tensor.to(device)
# optimizer = torch.optim.Adam(linear_model.parameters(), lr=0.01, weight_decay=5e-4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [22]:
def r2_loss(output, target):
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - output) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

In [23]:
# train fun
def train():
    model.train()
    optimizer.zero_grad()
    out = model(X_train_tensor)
    loss = F.mse_loss(out, y_train_tensor)
    loss.backward()
    optimizer.step()   
    return float(loss)

In [24]:
# test fun
@torch.no_grad()
def test():
    model.eval()
    train_pred = model(X_train_tensor)
    test_pred = model(X_test_tensor)
    
    train_mse = F.mse_loss(train_pred, y_train_tensor)
    test_mse = F.mse_loss(test_pred, y_test_tensor)
    train_r2 = r2_loss(train_pred, y_train_tensor)
    test_r2 = r2_loss(test_pred, y_test_tensor)
    return train_mse, test_mse, train_r2, test_r2
    

In [25]:
# Training the model
epochs = 10000
for epoch in range(1, epochs):
    loss = train()
    train_mse, test_mse, train_r2, test_r2 = test() 
    if epoch%100 == 0:
        print(f'Epoch: {epoch:03d}, Train Loss: {train_mse:.6f}, Train R2: {train_r2:.4f}, Test R2: {test_r2:.4f}')


Epoch: 100, Train Loss: 0.000081, Train R2: 0.5289, Test R2: 0.1795
Epoch: 200, Train Loss: 0.000073, Train R2: 0.5783, Test R2: 0.3153
Epoch: 300, Train Loss: 0.000067, Train R2: 0.6095, Test R2: 0.4010
Epoch: 400, Train Loss: 0.000064, Train R2: 0.6298, Test R2: 0.4549
Epoch: 500, Train Loss: 0.000061, Train R2: 0.6451, Test R2: 0.4903
Epoch: 600, Train Loss: 0.000059, Train R2: 0.6578, Test R2: 0.5151
Epoch: 700, Train Loss: 0.000057, Train R2: 0.6685, Test R2: 0.5335
Epoch: 800, Train Loss: 0.000056, Train R2: 0.6772, Test R2: 0.5473
Epoch: 900, Train Loss: 0.000054, Train R2: 0.6842, Test R2: 0.5576
Epoch: 1000, Train Loss: 0.000054, Train R2: 0.6895, Test R2: 0.5653
Epoch: 1100, Train Loss: 0.000053, Train R2: 0.6935, Test R2: 0.5708
Epoch: 1200, Train Loss: 0.000052, Train R2: 0.6965, Test R2: 0.5746
Epoch: 1300, Train Loss: 0.000052, Train R2: 0.6987, Test R2: 0.5773
Epoch: 1400, Train Loss: 0.000052, Train R2: 0.7002, Test R2: 0.5791
Epoch: 1500, Train Loss: 0.000051, Train R2

In [28]:
# save the performance
gcn_performance_dic['node-only'] = pd.DataFrame([train_r2.cpu(), test_r2.cpu()], 
                                                index = ['training', 'testing'],
                                                columns = ['node-only'])
gcn_performance_dic['node-only']


,node-only
training,0.708898
testing,0.589662


# GCN (Depth = 1): spatial, aggregate, strong, and weak networks

In [29]:
# create the baseline regressions
input_vars = ['pop_total_2016',
              'pop_density_2016',
              'age_median_2016', 
              'household_size_avg_2016',
              'sex_male_ratio_2016', 
              'race_white_ratio_2016',
              'edu_bachelor_ratio_2016',
              'edu_master_ratio_2016',
              'edu_master_ratio_2016', 
              'vehicle_per_capita_2016']

# predict inc_per_capita_2018
output_var = 'inc_per_capita_2018'

X = normalize(df_socioecon_shp[input_vars].values, axis = 0)
y = normalize(df_socioecon_shp[output_var].values.reshape(-1, 1), axis = 0).reshape(-1, 1)

X = X.astype('float32')
y = y.astype('float32')

X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y)


In [30]:
# creating the train and test masks
np.random.seed(seed = 42)
train_mask = np.random.choice([True, False], X_tensor.size()[0], p = [0.67,0.33]) #.reshape(-1, 1)
test_mask = ~train_mask
train_mask_tensor = torch.from_numpy(train_mask)
test_mask_tensor = torch.from_numpy(test_mask)

print(train_mask_tensor.sum())
print(test_mask_tensor.sum())

tensor(682)
tensor(310)


In [31]:
# start with the simplest case.
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(data.num_node_features, 1, bias = True)
        self.lin1 = Linear(-1, 1, bias = True)

    def forward(self):
        x, edge_index = data.x, data.edge_index
#         print(x, edge_index)
        x = self.conv1(x, edge_index) + self.lin1(x)
        return x

In [32]:
# r2 definition
def r2_loss(output, target):
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - output) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

In [33]:
# train function
def train():
    model.train()
    optimizer.zero_grad()
    out = model.forward()[data['train_mask']]
    loss = F.mse_loss(out, data.y[data['train_mask']])
    loss.backward()
    optimizer.step()   
    return float(loss)

In [34]:
# test function
@torch.no_grad()
def test():
    model.eval()
    train_pred = model.forward()[data['train_mask']]
    test_pred = model.forward()[data['test_mask']]
    
    train_mse = F.mse_loss(train_pred, data.y[data['train_mask']])
    test_mse = F.mse_loss(test_pred, data.y[data['test_mask']])
    train_r2 = r2_loss(train_pred, data.y[data['train_mask']])
    test_r2 = r2_loss(test_pred, data.y[data['test_mask']])    
    return train_mse, test_mse, train_r2, test_r2

In [35]:
# test four adj matrices
# note that the four matrices are quite simple examples. 

A_dic = {}
A_dic['spatial'] = utils.turn_df_to_adj(spatial_network_dic['boston_queen_contiguity_adj_df'], df_shp)
A_dic['spatial'] = A_dic['spatial'].values
A_dic['aggregate'] = utils.turn_df_to_adj(A_home_total_unweighted_dic[1.0]['total'], df_shp)
A_dic['aggregate'] = A_dic['aggregate'].values
A_dic['strong'] = utils.turn_df_to_adj(A_home_activity_unweighted_dic[1.0]['Office'], df_shp)
A_dic['strong'] = A_dic['strong'].values
A_dic['weak'] = utils.turn_df_to_adj(A_home_activity_unweighted_dic[1.0]['New American'], df_shp)
A_dic['weak'] = A_dic['weak'].values

# A = utils.turn_df_to_adj(A_home_activity_unweighted_dic[1.0]['New American'], df_shp)
# A = utils.turn_df_to_adj(A_home_activity_unweighted_dic[1.0]['Office'], df_shp)
# A = utils.turn_df_to_adj(spatial_network_dic['boston_queen_contiguity_adj_df'], df_shp)
# A = utils.turn_df_to_adj(A_home_total_unweighted_dic[1.0]['total'], df_shp)
# A = A.values

A_data_dic = {}

for key_ in A_dic.keys():
    A = A_dic[key_]
    # create the list of edges
    row_idx = csr_matrix(A).tocoo().row
    col_idx = csr_matrix(A).tocoo().col
    data_idx = csr_matrix(A).tocoo().data

    # edge index
    edge_index = torch.tensor([list(row_idx)+list(col_idx), list(col_idx)+list(row_idx)], dtype = torch.long)

    # init data
    data = Data(edge_index=edge_index, x = X_tensor, y = y_tensor, 
                train_mask = train_mask_tensor, 
                test_mask = test_mask_tensor)
    
    A_data_dic[key_] = data


In [36]:
# performance table
gcn_performance_one_layer_df = pd.DataFrame(np.zeros((2, 4)), 
                                            index = ['training', 'testing'], 
                                            columns = list(A_dic.keys()))

In [60]:
# model set up
for key_ in A_dic.keys():
    # list
    r2_train_list = []
    r2_test_list = []
    # device
    device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
    # data
    data = A_data_dic[key_]
    data = data.to(device)
    model = GCN().to(device)
    # optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-9)

    # Training the model 
    epochs = 10000
    for epoch in range(1, epochs):
        loss = train()
        train_mse, test_mse, train_r2, test_r2 = test()
        if epoch%100 == 0:
            print(f'Epoch: {epoch:03d}, Train Loss: {train_mse:.6f}, Train R2: {train_r2:.4f}, Test R2: {test_r2:.4f}')
            # 
            r2_train_list.append(float(train_r2.cpu().numpy()))
            r2_test_list.append(float(test_r2.cpu().numpy()))
    
    # Save the best testing r2 and corresponding training r2
    gcn_performance_one_layer_df.loc['testing', key_] = np.max(r2_test_list)
    idx = r2_test_list.index(np.max(r2_test_list))
    gcn_performance_one_layer_df.loc['training', key_] = r2_train_list[idx]


Epoch: 100, Train Loss: 0.000265, Train R2: -0.7716, Test R2: -0.6748
Epoch: 200, Train Loss: 0.000203, Train R2: -0.3559, Test R2: -0.3032
Epoch: 300, Train Loss: 0.000164, Train R2: -0.0946, Test R2: -0.0661
Epoch: 400, Train Loss: 0.000131, Train R2: 0.1219, Test R2: 0.1281
Epoch: 500, Train Loss: 0.000106, Train R2: 0.2935, Test R2: 0.2784
Epoch: 600, Train Loss: 0.000087, Train R2: 0.4211, Test R2: 0.3881
Epoch: 700, Train Loss: 0.000073, Train R2: 0.5114, Test R2: 0.4648
Epoch: 800, Train Loss: 0.000064, Train R2: 0.5729, Test R2: 0.5169
Epoch: 900, Train Loss: 0.000058, Train R2: 0.6138, Test R2: 0.5517
Epoch: 1000, Train Loss: 0.000054, Train R2: 0.6408, Test R2: 0.5752
Epoch: 1100, Train Loss: 0.000051, Train R2: 0.6590, Test R2: 0.5914
Epoch: 1200, Train Loss: 0.000049, Train R2: 0.6716, Test R2: 0.6033
Epoch: 1300, Train Loss: 0.000048, Train R2: 0.6809, Test R2: 0.6126
Epoch: 1400, Train Loss: 0.000047, Train R2: 0.6880, Test R2: 0.6201
Epoch: 1500, Train Loss: 0.000046, Tr

Epoch: 2100, Train Loss: 0.000044, Train R2: 0.7074, Test R2: 0.6405
Epoch: 2200, Train Loss: 0.000044, Train R2: 0.7083, Test R2: 0.6412
Epoch: 2300, Train Loss: 0.000044, Train R2: 0.7089, Test R2: 0.6417
Epoch: 2400, Train Loss: 0.000043, Train R2: 0.7094, Test R2: 0.6421
Epoch: 2500, Train Loss: 0.000043, Train R2: 0.7098, Test R2: 0.6424
Epoch: 2600, Train Loss: 0.000043, Train R2: 0.7101, Test R2: 0.6427
Epoch: 2700, Train Loss: 0.000043, Train R2: 0.7103, Test R2: 0.6429
Epoch: 2800, Train Loss: 0.000043, Train R2: 0.7105, Test R2: 0.6430
Epoch: 2900, Train Loss: 0.000043, Train R2: 0.7107, Test R2: 0.6431
Epoch: 3000, Train Loss: 0.000043, Train R2: 0.7108, Test R2: 0.6432
Epoch: 3100, Train Loss: 0.000043, Train R2: 0.7109, Test R2: 0.6433
Epoch: 3200, Train Loss: 0.000043, Train R2: 0.7110, Test R2: 0.6433
Epoch: 3300, Train Loss: 0.000043, Train R2: 0.7111, Test R2: 0.6434
Epoch: 3400, Train Loss: 0.000043, Train R2: 0.7112, Test R2: 0.6434
Epoch: 3500, Train Loss: 0.000043,

Epoch: 4100, Train Loss: 0.000043, Train R2: 0.7109, Test R2: 0.6627
Epoch: 4200, Train Loss: 0.000043, Train R2: 0.7098, Test R2: 0.6581
Epoch: 4300, Train Loss: 0.000043, Train R2: 0.7110, Test R2: 0.6626
Epoch: 4400, Train Loss: 0.000043, Train R2: 0.7111, Test R2: 0.6626
Epoch: 4500, Train Loss: 0.000045, Train R2: 0.6998, Test R2: 0.6417
Epoch: 4600, Train Loss: 0.000043, Train R2: 0.7112, Test R2: 0.6624
Epoch: 4700, Train Loss: 0.000043, Train R2: 0.7113, Test R2: 0.6624
Epoch: 4800, Train Loss: 0.000045, Train R2: 0.7006, Test R2: 0.6424
Epoch: 4900, Train Loss: 0.000043, Train R2: 0.7114, Test R2: 0.6623
Epoch: 5000, Train Loss: 0.000043, Train R2: 0.7115, Test R2: 0.6623
Epoch: 5100, Train Loss: 0.000044, Train R2: 0.7053, Test R2: 0.6486
Epoch: 5200, Train Loss: 0.000043, Train R2: 0.7116, Test R2: 0.6623
Epoch: 5300, Train Loss: 0.000043, Train R2: 0.7117, Test R2: 0.6622
Epoch: 5400, Train Loss: 0.000044, Train R2: 0.7083, Test R2: 0.6651
Epoch: 5500, Train Loss: 0.000043,

Epoch: 6100, Train Loss: 0.000039, Train R2: 0.7376, Test R2: 0.6921
Epoch: 6200, Train Loss: 0.000039, Train R2: 0.7380, Test R2: 0.6906
Epoch: 6300, Train Loss: 0.000039, Train R2: 0.7380, Test R2: 0.6903
Epoch: 6400, Train Loss: 0.000039, Train R2: 0.7380, Test R2: 0.6895
Epoch: 6500, Train Loss: 0.000039, Train R2: 0.7381, Test R2: 0.6898
Epoch: 6600, Train Loss: 0.000039, Train R2: 0.7381, Test R2: 0.6896
Epoch: 6700, Train Loss: 0.000039, Train R2: 0.7382, Test R2: 0.6894
Epoch: 6800, Train Loss: 0.000039, Train R2: 0.7382, Test R2: 0.6892
Epoch: 6900, Train Loss: 0.000039, Train R2: 0.7382, Test R2: 0.6886
Epoch: 7000, Train Loss: 0.000039, Train R2: 0.7382, Test R2: 0.6887
Epoch: 7100, Train Loss: 0.000039, Train R2: 0.7383, Test R2: 0.6886
Epoch: 7200, Train Loss: 0.000039, Train R2: 0.7383, Test R2: 0.6884
Epoch: 7300, Train Loss: 0.000039, Train R2: 0.7383, Test R2: 0.6884
Epoch: 7400, Train Loss: 0.000039, Train R2: 0.7383, Test R2: 0.6880
Epoch: 7500, Train Loss: 0.000039,

In [61]:
gcn_performance_one_layer_df

,spatial,aggregate,strong,weak
training,0.734623,0.708316,0.708007,0.732303
testing,0.685014,0.646084,0.666264,0.699321


In [65]:
# save the performance
gcn_performance_dic['homo & one-layer'] = gcn_performance_one_layer_df

# GCN (Hetero & Depth = 2)

Four scenarios
- Three spatial networks
- Three strong networks
- Three weak networks
- One spatial network + two weak networks


In [81]:
from torch_geometric.data import HeteroData

In [82]:
# create the baseline regressions
input_vars = ['pop_total_2016',
              'pop_density_2016',
              'age_median_2016', 
              'household_size_avg_2016',
              'sex_male_ratio_2016', 
              'race_white_ratio_2016',
              'edu_bachelor_ratio_2016',
              'edu_master_ratio_2016',
              'edu_master_ratio_2016', 
              'vehicle_per_capita_2016']

# predict inc_per_capita_2018
output_var = 'inc_per_capita_2018'

X = normalize(df_socioecon_shp[input_vars].values, axis = 0)
y = normalize(df_socioecon_shp[output_var].values.reshape(-1, 1), axis = 0).reshape(-1, 1)

X = X.astype('float32')
y = y.astype('float32')

X_tensor = torch.from_numpy(X)
y_tensor = torch.from_numpy(y)


In [83]:
# creating the train and test masks
np.random.seed(seed = 42)
train_mask = np.random.choice([True, False], X_tensor.size()[0], p = [0.67,0.33]) #.reshape(-1, 1)
test_mask = ~train_mask
train_mask_tensor = torch.from_numpy(train_mask)
test_mask_tensor = torch.from_numpy(test_mask)

print(train_mask_tensor.sum())
print(test_mask_tensor.sum())

tensor(682)
tensor(310)


In [152]:
# Create adjacency dictionary.
A_dic = {}
A_dic['spatial (3)'] = {}
A_dic['strong (3)'] = {}
A_dic['weak (3)'] = {}
A_dic['spatial & weak (1 + 2)'] = {}
A_dic['spatial & strong & weak (1 + 1 + 1)'] = {}

# specify the heterogeneous spatial networks
A_dic['spatial (3)']['queen'] = utils.turn_df_to_adj(spatial_network_dic['boston_queen_contiguity_adj_df'], df_shp)
A_dic['spatial (3)']['queen'] = A_dic['spatial (3)']['queen'].values
A_dic['spatial (3)']['rook'] = utils.turn_df_to_adj(spatial_network_dic['boston_rook_contiguity_adj_df'], df_shp)
A_dic['spatial (3)']['rook'] = A_dic['spatial (3)']['rook'].values
A_dic['spatial (3)']['5nn'] = utils.turn_df_to_adj(spatial_network_dic['boston_5nn_contiguity_adj_df'], df_shp)
A_dic['spatial (3)']['5nn'] = A_dic['spatial (3)']['5nn'].values

# specify the heterogeneous strong mobility networks
A_dic['strong (3)']['Office'] = utils.turn_df_to_adj(A_home_activity_unweighted_dic[1.0]['Office'], df_shp)
A_dic['strong (3)']['Office'] = A_dic['strong (3)']['Office'].values
A_dic['strong (3)']['Residential'] = utils.turn_df_to_adj(A_home_activity_unweighted_dic[1.0]['Residential'], df_shp)
A_dic['strong (3)']['Residential'] = A_dic['strong (3)']['Residential'].values
A_dic['strong (3)']['Building'] = utils.turn_df_to_adj(A_home_activity_unweighted_dic[1.0]['Building'], df_shp)
A_dic['strong (3)']['Building'] = A_dic['strong (3)']['Building'].values

# specify the heterogeneous weak mobility networks
A_dic['weak (3)']['New American'] = utils.turn_df_to_adj(A_home_activity_unweighted_dic[1.0]['New American'], df_shp)
A_dic['weak (3)']['New American'] = A_dic['weak (3)']['New American'].values
A_dic['weak (3)']['French'] = utils.turn_df_to_adj(A_home_activity_unweighted_dic[1.0]['French'], df_shp)
A_dic['weak (3)']['French'] = A_dic['weak (3)']['French'].values
A_dic['weak (3)']['Salad'] = utils.turn_df_to_adj(A_home_activity_unweighted_dic[1.0]['Salad'], df_shp)
A_dic['weak (3)']['Salad'] = A_dic['weak (3)']['Salad'].values

# specify the heterogeneous spatial + weak mobility networks
A_dic['spatial & weak (1 + 2)']['New American'] = utils.turn_df_to_adj(A_home_activity_unweighted_dic[1.0]['New American'], df_shp)
A_dic['spatial & weak (1 + 2)']['New American'] = A_dic['spatial & weak (1 + 2)']['New American'].values
A_dic['spatial & weak (1 + 2)']['French'] = utils.turn_df_to_adj(A_home_activity_unweighted_dic[1.0]['French'], df_shp)
A_dic['spatial & weak (1 + 2)']['French'] = A_dic['spatial & weak (1 + 2)']['French'].values
A_dic['spatial & weak (1 + 2)']['queen'] = utils.turn_df_to_adj(spatial_network_dic['boston_queen_contiguity_adj_df'], df_shp)
A_dic['spatial & weak (1 + 2)']['queen'] = A_dic['spatial & weak (1 + 2)']['queen'].values

# specify the heterogeneous spatial + strong + weak mobility networks
A_dic['spatial & strong & weak (1 + 1 + 1)']['New American'] = utils.turn_df_to_adj(A_home_activity_unweighted_dic[1.0]['New American'], df_shp)
A_dic['spatial & strong & weak (1 + 1 + 1)']['New American'] = A_dic['spatial & strong & weak (1 + 1 + 1)']['New American'].values
A_dic['spatial & strong & weak (1 + 1 + 1)']['Office'] = utils.turn_df_to_adj(A_home_activity_unweighted_dic[1.0]['Office'], df_shp)
A_dic['spatial & strong & weak (1 + 1 + 1)']['Office'] = A_dic['spatial & strong & weak (1 + 1 + 1)']['Office'].values
A_dic['spatial & strong & weak (1 + 1 + 1)']['queen'] = utils.turn_df_to_adj(spatial_network_dic['boston_queen_contiguity_adj_df'], df_shp)
A_dic['spatial & strong & weak (1 + 1 + 1)']['queen'] = A_dic['spatial & strong & weak (1 + 1 + 1)']['queen'].values


In [180]:
class HeteroGNN_1layer(torch.nn.Module):
    # damn it. The script sucks. I will rewrite it to freely specify the # of layers. 
    def __init__(self, metadata):
        super().__init__()
        self.conv1 = HeteroConv({edge_type: GCNConv(-1, 1, bias = True) for edge_type in metadata[1]})        
        self.lin1 = Linear(-1, 1) # 15 is the hidden channels. Feel free to change it. 

    def forward(self, x_dict, edge_index_dict):
        x_dict = self.conv1(x_dict, edge_index_dict)['ct'] + self.lin1(x_dict['ct'])
#         x_dict = {'ct':x_dict.relu()}
#         x_dict = self.conv2(x_dict, edge_index_dict)['ct'] + self.lin2(x_dict['ct'])
        return x_dict

In [144]:
class HeteroGNN_2layer(torch.nn.Module):
    # damn it. The script sucks. I will rewrite it to freely specify the # of layers. 
    def __init__(self, metadata):
        super().__init__()
        self.conv1 = HeteroConv({edge_type: GCNConv(-1, 15, bias = True) for edge_type in metadata[1]})
        self.conv2 = HeteroConv({edge_type: GCNConv(15, 1, bias = True) for edge_type in metadata[1]})
        
        self.lin1 = Linear(-1, 15) # 15 is the hidden channels. Feel free to change it. 
        self.lin2 = Linear(15, 1)

    def forward(self, x_dict, edge_index_dict):
        x_dict = self.conv1(x_dict, edge_index_dict)['ct'] + self.lin1(x_dict['ct'])
        x_dict = {'ct':x_dict.relu()}
        x_dict = self.conv2(x_dict, edge_index_dict)['ct'] + self.lin2(x_dict['ct'])
        return x_dict

In [161]:
class HeteroGNN_3layer(torch.nn.Module):
    # damn it. The script sucks. I will rewrite it to freely specify the # of layers. 
    def __init__(self, metadata):
        super().__init__()
        self.conv1 = HeteroConv({edge_type: GCNConv(-1, 15, bias = True) for edge_type in metadata[1]})
        self.conv2 = HeteroConv({edge_type: GCNConv(15, 15, bias = True) for edge_type in metadata[1]})
        self.conv3 = HeteroConv({edge_type: GCNConv(15, 1, bias = True) for edge_type in metadata[1]})
        
        self.lin1 = Linear(-1, 15) # 15 is the hidden channels. Feel free to change it. 
        self.lin2 = Linear(15, 15)
        self.lin3 = Linear(15, 1)

    def forward(self, x_dict, edge_index_dict):
        x_dict = self.conv1(x_dict, edge_index_dict)['ct'] + self.lin1(x_dict['ct'])
        x_dict = {'ct':x_dict.relu()}
        x_dict = self.conv2(x_dict, edge_index_dict)['ct'] + self.lin2(x_dict['ct'])
        x_dict = {'ct':x_dict.relu()}
        x_dict = self.conv3(x_dict, edge_index_dict)['ct'] + self.lin3(x_dict['ct'])
        return x_dict

In [169]:
class HeteroGNN_4layer(torch.nn.Module):
    # damn it. The script sucks. I will rewrite it to freely specify the # of layers. 
    def __init__(self, metadata):
        super().__init__()
        self.conv1 = HeteroConv({edge_type: GCNConv(-1, 15, bias = True) for edge_type in metadata[1]})
        self.conv2 = HeteroConv({edge_type: GCNConv(15, 15, bias = True) for edge_type in metadata[1]})
        self.conv3 = HeteroConv({edge_type: GCNConv(15, 15, bias = True) for edge_type in metadata[1]})
        self.conv4 = HeteroConv({edge_type: GCNConv(15, 1, bias = True) for edge_type in metadata[1]})
        
        self.lin1 = Linear(-1, 15) # 15 is the hidden channels. Feel free to change it. 
        self.lin2 = Linear(15, 15)
        self.lin3 = Linear(15, 15)
        self.lin4 = Linear(15, 1)

    def forward(self, x_dict, edge_index_dict):
        x_dict = self.conv1(x_dict, edge_index_dict)['ct'] + self.lin1(x_dict['ct'])
        x_dict = {'ct':x_dict.relu()}
        x_dict = self.conv2(x_dict, edge_index_dict)['ct'] + self.lin2(x_dict['ct'])
        x_dict = {'ct':x_dict.relu()}
        x_dict = self.conv3(x_dict, edge_index_dict)['ct'] + self.lin3(x_dict['ct'])
        x_dict = {'ct':x_dict.relu()}
        x_dict = self.conv4(x_dict, edge_index_dict)['ct'] + self.lin4(x_dict['ct'])

        return x_dict

In [181]:
# r2 definition
def r2_loss(output, target):
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - output) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

In [182]:
# define the train fun
def train():
    model.train()
    optimizer.zero_grad()
    out = model.forward(data.x_dict, data.edge_index_dict) 
    mask = data['ct'].train_mask
    loss = F.mse_loss(out[mask], data['ct'].y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)


In [183]:
# define the test fun
def test():
    model.eval()
    out = model.forward(data.x_dict, data.edge_index_dict)
    
    train_pred = out[data['ct'].train_mask]
    test_pred = out[data['ct'].test_mask]
    
    train_mse = F.mse_loss(train_pred, data['ct'].y[data['ct'].train_mask])
    test_mse = F.mse_loss(test_pred, data['ct'].y[data['ct'].test_mask])
    train_r2 = r2_loss(train_pred, data['ct'].y[data['ct'].train_mask])
    test_r2 = r2_loss(test_pred, data['ct'].y[data['ct'].test_mask])
    return train_mse, test_mse, train_r2, test_r2


In [184]:
# init data
data = HeteroData()

# init nodes
data['ct'].x = X_tensor
data['ct'].y = y_tensor
data['ct'].train_mask = train_mask_tensor
data['ct'].test_mask = test_mask_tensor

# add the hetero edges
for key_ in A_dic['spatial & weak (1 + 2)'].keys():
    # create the adj matrix
    A = A_dic['spatial & weak (1 + 2)'][key_]

#     # remove the self loop
#     np.fill_diagonal(A, 0.0)

    # create the list of edges
    row_idx = csr_matrix(A).tocoo().row
    col_idx = csr_matrix(A).tocoo().col
    data_idx = csr_matrix(A).tocoo().data

    # edge index
    edge_index = torch.tensor([list(row_idx)+list(col_idx), list(col_idx)+list(row_idx)], dtype = torch.long)
    data['ct', key_, 'ct'].edge_index = edge_index


In [185]:
data

HeteroData(
  ct={
    x=[992, 10],
    y=[992, 1],
    train_mask=[992],
    test_mask=[992]
  },
  (ct, New American, ct)={ edge_index=[2, 18316] },
  (ct, French, ct)={ edge_index=[2, 5012] },
  (ct, queen, ct)={ edge_index=[2, 11504] }
)

In [187]:
# init model
model = HeteroGNN_1layer(data.metadata())
# model = HeteroGNN_2layer(data.metadata())
# model = HeteroGNN_3layer(data.metadata())
# model = HeteroGNN_4layer(data.metadata())
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
data, model = data.to(device), model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-9) # need to readjust these parameters. 


In [188]:
### 
epochs = 15001
for epoch in range(1, epochs):
    loss = train()
    if epoch%500 == 0:
        train_mse, test_mse, train_r2, test_r2 = test()
        print(f'Epoch: {epoch:03d}, Train Loss: {train_mse:.6f}, Train R2: {train_r2:.4f}, Test R2: {test_r2:.4f}')


Epoch: 500, Train Loss: 0.000075, Train R2: 0.4999, Test R2: 0.4184
Epoch: 1000, Train Loss: 0.000051, Train R2: 0.6574, Test R2: 0.5831
Epoch: 1500, Train Loss: 0.000043, Train R2: 0.7106, Test R2: 0.6302
Epoch: 2000, Train Loss: 0.000039, Train R2: 0.7394, Test R2: 0.6592
Epoch: 2500, Train Loss: 0.000036, Train R2: 0.7562, Test R2: 0.6791
Epoch: 3000, Train Loss: 0.000035, Train R2: 0.7659, Test R2: 0.6931
Epoch: 3500, Train Loss: 0.000034, Train R2: 0.7713, Test R2: 0.7029
Epoch: 4000, Train Loss: 0.000034, Train R2: 0.7742, Test R2: 0.7095
Epoch: 4500, Train Loss: 0.000034, Train R2: 0.7700, Test R2: 0.7024
Epoch: 5000, Train Loss: 0.000033, Train R2: 0.7765, Test R2: 0.7152
Epoch: 5500, Train Loss: 0.000033, Train R2: 0.7767, Test R2: 0.7172
Epoch: 6000, Train Loss: 0.000033, Train R2: 0.7772, Test R2: 0.7165
Epoch: 6500, Train Loss: 0.000033, Train R2: 0.7771, Test R2: 0.7176
Epoch: 7000, Train Loss: 0.000033, Train R2: 0.7775, Test R2: 0.7165
Epoch: 7500, Train Loss: 0.000033, 

In [155]:
gcn_performance_two_layer_df = pd.DataFrame(np.zeros((2, 5)), 
                                           index = ['training', 'testing'],
                                           columns = list(A_dic.keys()))

In [159]:
# 
gcn_performance_two_layer_df.loc[['training', 'testing'], 'spatial (3)'] = 0.818, 0.735
gcn_performance_two_layer_df.loc[['training', 'testing'], 'strong (3)'] = 0.855, 0.688
gcn_performance_two_layer_df.loc[['training', 'testing'], 'weak (3)'] = 0.893, 0.751
gcn_performance_two_layer_df.loc[['training', 'testing'], 'spatial & weak (1 + 2)'] = 0.876, 0.770
gcn_performance_two_layer_df.loc[['training', 'testing'], 'spatial & strong & weak (1 + 1 + 1)'] = 0.811, 0.732


In [160]:
gcn_performance_two_layer_df

,spatial (3),strong (3),weak (3),spatial & weak (1 + 2),spatial & strong & weak (1 + 1 + 1)
training,0.818,0.855,0.893,0.876,0.811
testing,0.735,0.688,0.751,0.770,0.732


Depth:
one layer: 0.777, 0.716
two layer: 0.876, 0.770
three layer: 0.870, 0.544
four layer: 0.907, 0.720